In [ ]:
%matplotlib inline
import holoviews as hv
hv.notebook_extension()
import sympy.interactive
sympy.interactive.init_printing('mathjax')

import kwant
import numpy as np
from itertools import product
from supercurrent import (current_at_phase,
                          discretized_hamiltonian,
                          hoppingkind_at_interface,
                          hoppingkind_in_shape,
                          make_params,
                          peierls,
                          SimpleNamespace)


# 3D test system

In [ ]:
from supercurrent import k_B
sx_ = np.array([[0., 1.], [1., 0.]])
sz_ = np.array([[1., 0.], [0., -1.]])

onsite = lambda site, p: -p.mu * sz_
onsite_lead = lambda site, p: -p.mu * sz_ + p.delta * sx_
hops = lambda site1, site2, p: -p.t * sz_

def make_test_system(X, Y, Z):
    lat = kwant.lattice.general(np.eye(3))
    syst = kwant.Builder()
    range_X = range(X)
    range_Y = range(Y)
    range_Z = range(Z)
    syst[(lat(x, y, z) for x in range_X for y in range_Y for z in range_Z)] = onsite
    syst[lat.neighbors()] = hops

    l_cut = [lat(x, y, z) for x, y, z in [i.tag for i in syst.sites()] if x == 0]
    r_cut = [lat(x, y, z) for x, y, z in [i.tag for i in syst.sites()] if x == 1]

    num_orbs = 2
    dim = num_orbs * (len(l_cut) + len(r_cut))
    vlead = kwant.builder.SelfEnergyLead(lambda energy, args: np.zeros((dim, dim)), r_cut + l_cut)
    syst.leads.append(vlead)
    
    lead = kwant.Builder(kwant.TranslationalSymmetry((-1, 0, 0)))
    lead[(lat(0, y, z) for y in range_Y for z in range_Z)] = onsite_lead
    lead[lat.neighbors()] = hops
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    syst = syst.finalized()
    
    r_cut_sites = [syst.sites.index(site) for site in r_cut]
    l_cut_sites = [syst.sites.index(site) for site in l_cut]
    
    def hopping(syst, args=()):
        return syst.hamiltonian_submatrix(args=args,
                                          to_sites=l_cut_sites,
                                          from_sites=r_cut_sites)[::2, ::2]
    return syst, hopping



delta = 0.01
T = 0.5 * delta / k_B
syst, hopping = make_test_system(X=3, Y=3, Z=3)
p = SimpleNamespace(mu=0., t=1., delta=1)

phases = np.linspace(-np.pi, np.pi, 51)
H_0_cache = []
I = np.array([current_at_phase(syst, hopping, p, T, H_0_cache, phase, tol=None, max_frequencies=200) for phase in phases])
hv.Curve(I, kdims=['phase'], vdims=['$I$'])

# 3D system with Discretizer Hamiltonian, squared system

In [ ]:
def make_test_system(X, Y, Z, a=10):
    tb_normal, tb_sc = discretized_hamiltonian(a)[:-1]
    lat = tb_normal.lattice
    syst = kwant.Builder()
    syst[(lat(x, y, z) for x in range(X) for y in range(Y) for z in range(Z))] = tb_normal.onsite

    for hop, val in tb_normal.hoppings.items():
        syst[hop] = val

    l_cut = [lat(x, y, z) for x, y, z in [i.tag for i in syst.sites()] if x == 0]
    r_cut = [lat(x, y, z) for x, y, z in [i.tag for i in syst.sites()] if x == 1]

    num_orbs = 4
    dim = num_orbs * (len(l_cut) + len(r_cut))
    vlead = kwant.builder.SelfEnergyLead(lambda energy, args: np.zeros((dim, dim)), r_cut + l_cut)
    syst.leads.append(vlead)
    
    lead = kwant.Builder(kwant.TranslationalSymmetry((-a, 0, 0)))
    lead[(lat(0, y, z) for y in range(Y) for z in range(Z))] = tb_sc.onsite

    for hop, val in tb_sc.hoppings.items():
        lead[hop] = val

    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    
    syst = syst.finalized()
    
    r_cut_sites = [syst.sites.index(site) for site in r_cut]
    l_cut_sites = [syst.sites.index(site) for site in l_cut]
    
    def hopping(syst, args=()):
        return syst.hamiltonian_submatrix(args=args,
                                          to_sites=l_cut_sites,
                                          from_sites=r_cut_sites)[::2, ::2]
    return syst, hopping

T = 60e-3

syst, hopping = make_test_system(X=2, Y=3, Z=3)
p = make_params(Delta=0.25, alpha=20, mu=15)
phases = np.linspace(-np.pi, np.pi, 51)
H_0_cache = []
I = np.array([current_at_phase(syst, hopping, p, T, H_0_cache, phase, tol=None, max_frequencies=200) for phase in phases])
hv.Curve(I, kdims=['phase'], vdims=['$I$'])

# Simpler way (but slightly slower)

In [ ]:
from supercurrent import matsubara_frequency, k_B
def null_G(syst, p, T, n):
    en = matsubara_frequency(T, n)
    gf = kwant.greens_function(syst, en, [p], [0], [0], check_hermiticity=False)
    return gf.data[::2, ::2]

def current_from_G_0(T, G_0s, H12, phase):
    t = H12 * np.exp(1j * phase)
    dim = t.shape[0]
    I = 0
    for G_0 in G_0s:
        V = np.zeros_like(G_0, dtype=complex)
        v = t - H12
        V[:dim, dim:] = v.T.conj()
        V[dim:, :dim] = v
        gf = np.linalg.solve(np.identity(2*dim) - G_0 @ V, G_0)
        H12G21 = t.T.conj() @ gf[dim:, :dim]
        H21G12 = t @ gf[:dim, dim:]
        I += -4 * T * (np.trace(H21G12) - np.trace(H12G21)).imag
    return I

matsfreqs = 200
G_0s = [null_G(syst, p, T, n) for n in range(matsfreqs)]
H12 = hopping(syst, args=[p])
phases = np.linspace(-np.pi, np.pi, 51)
I = [current_from_G_0(T, G_0s, H12, phase) for phase in phases]
I = np.array(I)
hv.Curve((phases, I), kdims=['phase'], vdims=['$I$'])

# Test 2D system

In [ ]:
sx_ = np.array([[0., 1.], [1., 0.]])
sz_ = np.array([[1., 0.], [0., -1.]])

onsite = lambda site, p: -p.mu * sz_
onsite_lead = lambda site, p: -p.mu * sz_ + p.delta * sx_
hops = lambda site1, site2, p: -p.t * sz_

def make_test_system_2D(X=3, Y=2):
    lat = kwant.lattice.square()
    syst = kwant.Builder()
    range_X = range(X)
    range_Y = range(Y)
    syst[(lat(x, y) for x in range_X for y in range_Y)] = onsite
    syst[lat.neighbors()] = hops

    l_cut = [lat(x, y) for x, y in [i.tag for i in syst.sites()] if x == 0]
    r_cut = [lat(x, y) for x, y in [i.tag for i in syst.sites()] if x == 1]

    num_orbs = 2
    dim = num_orbs * (len(l_cut) + len(r_cut))
    vlead = kwant.builder.SelfEnergyLead(lambda energy, args: np.zeros((dim, dim)), r_cut + l_cut)
    syst.leads.append(vlead)
    
    lead = kwant.Builder(kwant.TranslationalSymmetry((-1, 0)))
    lead[(lat(0, y) for y in range_Y)] = onsite_lead
    lead[lat.neighbors()] = hops
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    syst = syst.finalized()
    
    r_cut_sites = [syst.sites.index(site) for site in r_cut]
    l_cut_sites = [syst.sites.index(site) for site in l_cut]
    
    def hopping(syst, args=()):
        return syst.hamiltonian_submatrix(args=args,
                                          to_sites=l_cut_sites,
                                          from_sites=r_cut_sites)[::2, ::2]
    return syst, hopping


In [ ]:
syst, _ = make_test_system_2D()
kwant.plot(syst)